In [5]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report, roc_auc_score

# ---------------------- 1. 데이터 로딩 및 전처리 ---------------------- #
train_transaction = pd.read_csv("train_transaction.csv")
train_identity = pd.read_csv("train_identity.csv")

df = pd.merge(train_transaction, train_identity, on="TransactionID", how="left")
df.fillna(0, inplace=True)

y = df["isFraud"].values
X = df.select_dtypes(include=[np.number]).drop(columns=["TransactionID", "isFraud"])

# 정규화 및 차원축소
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=20)
X_scaled = pca.fit_transform(X_scaled)

# ---------------------- 2. LF²-LOF 함수 정의 ---------------------- #
def lf2_lof_vectorized(X, n_neighbors=20, epsilon=1e-6):
    nbrs = NearestNeighbors(n_neighbors=n_neighbors + 1).fit(X)
    _, indices = nbrs.kneighbors(X)
    neighbors = X[indices[:, 1:]]
    X_expanded = X[:, np.newaxis, :]
    abs_diff = np.abs(X_expanded - neighbors)
    mean_abs_diff = np.mean(abs_diff, axis=1)
    std_dev = np.std(neighbors, axis=1) + epsilon
    scores = np.mean(mean_abs_diff / std_dev, axis=1)
    return scores

# ---------------------- 3. 이상치 점수 계산 및 평가 ---------------------- #
# 상위 5%를 이상치로 간주
threshold = np.percentile(lf2_scores, 95)
y_pred = (lf2_scores > threshold).astype(int)

# 비지도 평가
print("📊 평가 결과:")
print(classification_report(y, y_pred, target_names=["Not Fraud", "Fraud"]))
print(f"🎯 ROC AUC (Unsupervised): {roc_auc_score(y, lf2_scores):.4f}")


📊 평가 결과:
              precision    recall  f1-score   support

   Not Fraud       0.98      0.95      0.97      1469
       Fraud       0.08      0.19      0.11        31

    accuracy                           0.94      1500
   macro avg       0.53      0.57      0.54      1500
weighted avg       0.96      0.94      0.95      1500

🎯 ROC AUC (Unsupervised): 0.7361
